# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model
- Compare and contrast model validation strategies

## Let's Get Started

We included the code to pre-process the Ames Housing dataset below. This is done for the sake of expediency, although it may result in data leakage and therefore overly optimistic model metrics.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

## Train-Test Split

Perform a train-test split with a test set of 20% and a random state of 4.

In [3]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split


In [19]:
# Split the data into training and test sets (assign 20% to test set)
x_train,x_test,y_train,y_test= train_test_split(X,y,random_state=4,test_size=0.2)
x_train,x_test,y_train,y_test

(      LotArea_log  1stFlrSF_log  GrLivArea_log  BldgType_2fmCon  \
 666      1.375948      0.887889       1.520254                0   
 226      0.182607      0.296398       1.691618                0   
 155      0.113403     -2.072081      -0.804532                0   
 231      0.993578      0.941145       2.001062                0   
 901     -0.074175      0.526305      -0.278969                0   
 ...           ...           ...            ...              ...   
 360     -0.353390     -0.603850      -1.355520                0   
 709     -0.452787     -0.631580      -1.381935                0   
 439      0.600827     -1.509279      -0.542756                0   
 174      0.610502      1.264076       0.423810                0   
 1146     0.411309      0.506966      -0.297390                0   
 
       BldgType_Duplex  BldgType_Twnhs  BldgType_TwnhsE  KitchenQual_Fa  \
 666                 0               0                0               0   
 226                 0          

### Fit a Model

Fit a linear regression model on the training set

In [6]:
# Import LinearRegression from sklearn.linear_model
from sklearn.linear_model import LinearRegression

model= LinearRegression()



In [8]:
# Instantiate and fit a linear regression model
model.fit(x_train,y_train)
y_test_pred=model.predict(x_test)
y_train_pred=model.predict(x_train)

### Calculate MSE

Calculate the mean squared error on the test set

In [10]:
# Import mean_squared_error from sklearn.metrics
from sklearn.metrics import mean_squared_error


In [11]:
# Calculate MSE on test set
train_mse=mean_squared_error(y_train,y_train_pred)
test_mse=mean_squared_error(y_test,y_test_pred)


## Cross-Validation using Scikit-Learn

Now let's compare that single test MSE to a cross-validated test MSE.

In [13]:
# Import cross_val_score from sklearn.model_selection
from sklearn.model_selection import cross_val_score



In [16]:
# Find MSE scores for a 5-fold cross-validation
five_cv=cross_val_score(model,X,y)
five_cv

array([0.86724871, 0.82916275, 0.82017954, 0.81167639, 0.78362276])

In [18]:
# Get the average MSE score
mean_cv=np.mean(five_cv)
mean_cv

0.8223780298511537

Compare and contrast the results. What is the difference between the train-test split and cross-validation results? Do you "trust" one more than the other?

In [ ]:
# Your answer here


## Level Up: Let's Build It from Scratch!

### Create a Cross-Validation Function

Write a function `kfolds(data, k)` that splits a dataset into `k` evenly sized pieces. If the full dataset is not divisible by `k`, make the first few folds one larger then later ones.

For example, if you had this dataset:

In [22]:
example_data = pd.DataFrame({
    "color": ["red", "orange", "yellow", "green", "blue", "indigo", "violet"]
})
example_data

,color
0,red
1,orange
2,yellow
3,green
4,blue
5,indigo
6,violet


`kfolds(example_data, 3)` should return:

* a dataframe with `red`, `orange`, `yellow`
* a dataframe with `green`, `blue`
* a dataframe with `indigo`, `violet`

Because the example dataframe has 7 records, which is not evenly divisible by 3, so the "leftover" 1 record extends the length of the first dataframe.

In [30]:
def kfolds(data, k):
    folds = []
    
    # Your code here
    n=len(data)
    fold_size= n//k
    reminder = n % k
    start=0
    for i in range(k):
        end = start + fold_size + (1 if i < reminder else 0)
        fold = data.iloc[start:end]
        folds.append(fold)
        start=end
    

        

    
    return folds

In [31]:
results = kfolds(example_data, 3)
for result in results:
    print(result, "\n")

    color
0     red
1  orange
2  yellow 

   color
3  green
4   blue 

    color
5  indigo
6  violet 



### Apply Your Function to the Ames Housing Data

Get folds for both `X` and `y`.

In [ ]:
# Apply kfolds() to ames_data with 5 folds


### Perform a Linear Regression for Each Fold and Calculate the Test Error

Remember that for each fold you will need to concatenate all but one of the folds to represent the training data, while the one remaining fold represents the test data.

In [ ]:
# Replace None with appropriate code
test_errs = []
k = 5

for n in range(k):
    # Split into train and test for the fold
    X_train = None
    X_test = None
    y_train = None
    y_test = None
    
    # Fit a linear regression model
    None
    
    # Evaluate test errors
    None

print(test_errs)

If your code was written correctly, these should be the same errors as scikit-learn produced with `cross_val_score` (within rounding error). Test this out below:

In [ ]:
# Compare your results with sklearn results


This was a bit of work! Hopefully you have a clearer understanding of the underlying logic for cross-validation if you attempted this exercise.

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are more robust than train-test split.